# Q3: A 2 meter long beam has a linear mass density $\lambda(x) = x^2$,  where x is measured from one its ends. Find the center of mass of the beam numerically.

I have solved this problem in two methods

In [1]:
from library.integration import simpson_rule
from library.matrix import truncate_p as truncate

## A. Naive approach:
Here I (as a human) have to calculate a lot of things and simplify it to a single expression. <br>
This approach guarantees that the result is correct and deterministically reaches the answer. <br>
But here, the computer does not do everything for us, we have to do a lot of calculations ourselves.

The formula for the center of mass is:

$$x_{COM} = \frac{\sum m_i x_i}{\sum m_i} = \frac{\int_0^M x dm}{M}\;\;\;\;\;[M = total\;mass]$$

now, mass density is given to be $\lambda(x) = x^2$, so the center of mass will be at:

$$x_{COM} = \frac{\int \lambda x dx}{\int xdx}$$

Putting the value of $\lambda(x)=x^2$ in the above equation, we get:

$$x_{COM} = \frac{\int_0^2 x^3 dx}{\int_0^2 x^2dx}$$



In [2]:
lower_limit = 0
upper_limit = 2
steps = 3
a = simpson_rule(lambda x: x**3, lower_limit, upper_limit, steps)
b = simpson_rule(lambda x: x**2, lower_limit, upper_limit, steps)
x_COM = a/b
print(f"Center of mass is at x = {x_COM}m")
# value was supposed to come out to be x = 1.5m.
# The small error can be avoided if we took more steps

Center of mass is at x = 1.4999999999999998m


## B. More intuitive approach

Lazy me wanted my computer to do all the work

### Aim:
&emsp;To find the center of mass of the beam with mass density $\lambda(x) = x^2$.

### Theory:
&emsp;Let us use the fact that, if a body is pivoted at its center of mass, it will not rotate (net torque on it will be zero). So, we will try to find the net torque on the body if pivoted at a random position $a$. Then we will minimise the net torque (minimum value will be close to 0) to find the center of mass. This idea is inspired from the linear regression algorithm.

### Method:
 - start at a random location on the beam (we have started with $a = 0$)
 - find torque on the beam if pivoted at that location (point $a$)
 - change the value of $a$ depending on the torque (if torque is positive, move a to the right, if negative, move a to the left. Also the distance by which we move $a$ is proportional (proportionality constant is $\alpha$) to the torque)
 - with every step, we will reach a point where the torque will be lesser and lesser. Thus we will keep on taking smaller and smaller steps until we reach a point where the torque is almost zero. This point will be the center of mass of the beam. (**Note:** If we take a large value of $\alpha$, the algorithm may not converge to the correct answer, rather it will diverge out. If we take a small value of $\alpha$, the algorithm will take a lot of time to converge to the correct answer. So, we have to choose a value of $\alpha$ such that the algorithm converges to the correct answer in a reasonable amount of time.)

In [3]:
def get_torque(a):
	f_wrt_a = lambda x: (x-a)*x**2
	torque_left = simpson_rule(f_wrt_a,  0,  a,  n=4)
	torque_right = simpson_rule(f_wrt_a,  a,  2,  n=4)
	return torque_left + torque_right

In [4]:
get_torque(2.5)

-2.666666666666667

In [38]:
tollerance = 1e-6
func = lambda x: x**2
alpha = 0.1  # we have carefully chosen this value such that:
# the value of 'a' overshoots the center of mass with every leap.
# This way it would be easier to visualise it. My main aim was
# not to find the center of mass in the lowest number of steps.
# to find the center of mass in the lowest possible steps, take
# smaller values of a (maybe a = 0.1)

net_torque = 1  # initiate at any value more than tollerance
a = 0  # initiate at any point on the rod (0, 2)... although it doesn't matter
steps = 0  # for counting the steps
found = True  

net_torques = []
positions = []
while abs(net_torque)>tollerance:
    positions.append(a)  # just keeping the records
    # print(f"{steps}:\ta = {a:.6f},", end = "\t")
    f_wrt_a = lambda x: func(x)*(x-a)
    torque_left  = simpson_rule(f_wrt_a,  0,  a,  n=2)
    torque_right = simpson_rule(f_wrt_a,  a,  2,  n=2)
    # simpson rule acts so good here because the f_wrt_a is a tertiary function
    # and simpson rule is based upon quadratic model... 2 and 3 are close numbers
    net_torque = torque_left + torque_right
    net_torques.append(abs(net_torque))  # just keeping the records
    a += net_torque*alpha  # alpha is a constant, it acts as the regulator, how fast a changes
    steps += 1
    # print(f"net_torque = {net_torque:.6f}")
    if len(net_torques)-1 and net_torques[-1] > net_torques[-2]:
        print("The net_torque is increasing, the algorithm is diverging, decreasing alpha might help!")
        found = False
        break

if found: print(f"Center of mass is at x = {truncate(a, 6, str)}m")

Center of mass is at x = 1.499999m


### Demonstration:

&emsp; Dotted lines are only for directing your eyes to the point of interest. The magenta dotted line is the variation of absolute value of torque with $a$. The cyan dotted line denotes the change in $a$ with each iteration. **(points on the of the cyan line don't  mean anything)**

&emsp; Although I am not backing my claim with mathematically rigorous proof, but from the graph we can see that the variation of torque with $a$ is a linear curve with slope $-\frac{8}{3}$ and the curve intersects the x-axis at $a = 1.5$. So, the center of mass is at $a = 1.5$.

#### Here are some graphs for different values of $\alpha$:
(To see the code to generate this graph, visit [here](https://gist.github.com/PeithonKing/dcf64c551cf265cf7ed20901beffe150))
![alpha = 0.65](static/ass5_1.png)
![alpha = 0.1](static/ass5_2.png)
![alpha = 0.8](static/ass5_3.png)

In [ ]:
import matplotlib.pyplot as plt
from library.matrix import linspace
g1 = lambda x: abs(-(8/3)*x+4)

# PLOTTING
xs = linspace(0, 2.6, 100)
plt.plot(xs, [g1(xs) for xs in xs], "m--")  # visual aid 1
plt.plot(positions, net_torques, "c--")  # visual aid 2: line joining points
plt.scatter(positions[1:-1], net_torques[1:-1], c="#000000")  # points
plt.scatter(positions[0], net_torques[0], c="g")  
plt.scatter(positions[-1], net_torques[-1], c="r")

# annotating points
try:
    plt.annotate("1, start", (positions[0]+0.05, net_torques[0]))
    for i in range(6):
        offset = -0.1 if i%2 else 0.05
        plt.annotate(f"{i+2}", (positions[i+1]+offset, net_torques[i+1]-0.1))
    plt.annotate("and so on...", (positions[7]+0.05, net_torques[7]-0.1))
    plt.annotate(f"{len(positions)}, end", (positions[-1]+0.05, net_torques[-1]-0.1))
except IndexError: pass

# Legends and Labels
plt.legend(["visual aid 1", "visual aid 2", "points"])
plt.xlabel("positions")
plt.ylabel("absolute value of net torques")
plt.text(0.25, 1.5, f"$\\alpha$ = {alpha}", fontsize=12)
if not found: plt.text(0.25, 1, f"Didn't converge", fontsize=12)